Import Classes and Define Variables

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pymongo
import pandas as pd
import nsepy
from datetime import datetime
from datetime import timedelta
from datetime import date
from pymongo import MongoClient
from nsepy import get_history
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]


8425

Data Collector Function

In [2]:
def dataCollector():
    equityListPd = pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"}))
    collectionTime=2000
    basicSample=30
    cnt=0
    my_time = datetime.now()

    for rows in equityListPd['Issuer Name'].iteritems():
        print(cnt,rows[1],end="-")
        cnt= cnt + 1
        if stockWarehouse.find({'Symbol':rows[1]}).count() > 1 :
            for j in stockWarehouse.find({'Symbol':rows[1]},{'_id':0,'Date':1}).sort('Date',-1).limit(1) :
                collectionTime=((my_time - j['Date']).days - 1)
                fromDate=j['Date']
        if fromDate.date() < datetime.now().date():
            
            iters=int(collectionTime/basicSample)
            if (collectionTime != 0):
                if collectionTime < 2000:
                    basicSample=collectionTime
                    iters=1
                dt=datetime.now()
                print("Iterations:",iters,end="|")
                for i in range(0,iters):

                    startDate=dt - timedelta(days=basicSample)
                    if( startDate <= fromDate):
                        startDate = (fromDate + timedelta(days=1))
                    endDate=dt 
                    print(i,"Collecting into between",startDate.date(),"-", endDate.date(),"@",datetime.now(),end="|")
                    df=pd.DataFrame(get_history(symbol=rows[1] , start= startDate,end=endDate))
                    df.reset_index(inplace=True)
                    df['Date']=pd.to_datetime(df['Date'])
                    dfLength=(len(df.index))
                    if  dfLength > 0 :
                        data_dict=df.to_dict("records")
                        #print(data_dict)
                        stockWarehouse.insert_many(data_dict)
                        tmpdf=pd.DataFrame(stockWarehouse.find({"Symbol":rows[i],"Date":{"$lte":endDate,"$gte":startDate}}))
                        print(df.index.size,tmpdf.index.size,end="|")

                    else :
                        print("Marking as Inactive")
                        equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Inactive'}})
                        break;
                    dt=(startDate - timedelta(days=1))


In [3]:
def markInactiveStocks():
    reference=int(stockWarehouse.find({'Symbol':'SBIN'}).count()) - 100
    print(reference)
    tmp_count=0
    for i in equityList.find({},{'Issuer Name':1}):
        #print(i['Issuer Name'])
        tmp_count=(int(stockWarehouse.find({'Symbol':i['Issuer Name']}).count()))
        if tmp_count >= reference:
            print(i['Issuer Name'],end='|')
            equityList.update_many({'Issuer Name':i['Issuer Name']},{'$set': {'State':"A"}})

In [4]:
def markTrend_Crossover(stockdf,datadf,symbol):
    
    for i in range(0,stockdf.shape[0],1):
            if stockdf['Issuer Name'].iloc[i]==symbol:
                break;
    datadf['ewmdiff']=datadf['Close'].ewm(span=9,adjust=False).mean() - datadf['Close'].ewm(span=21,adjust=False).mean()
    nrows=datadf.shape[0]-1
    if datadf.iloc[nrows,15] > 0:
        stockdf.iloc[i,1]=1
    else:
        stockdf.iloc[i,1]=-1
    for j in range(datadf.shape[0]-1,2,-1):
        if datadf.iloc[j,15]*datadf.iloc[j-1,15] <= 0:
            stockdf.iloc[i,2]=datadf.shape[0]-j
            break;
    datadf['ewmdiffewm14']=datadf['ewmdiff'].ewm(span=9,adjust=False).mean()
    if datadf.iloc[nrows,15] > datadf.iloc[nrows,16] :
        stockdf.iloc[i,3]='Expanding'
    else:
        stockdf.iloc[i,3]='Contracting'
    return(stockdf)

In [5]:
def singlePatternCandle(stockdf,datadf,symbol):
    for i in range(0,stockdf.shape[0],1):
        if stockdf['Issuer Name'].iloc[i]==symbol:
            break;
                
    pattern=''
    umbrellaFound=False
    if 'singleCandlePattern' in stockdf.columns:
        if pd.isnull(stockdf['singleCandlePattern'].iloc[i]):
            pattern=''
        else:
            pattern=stockdf['singleCandlePattern'].iloc[i]
    nrows=datadf.shape[0]-1
    Close=datadf['Close'].iloc[nrows]
    Open=datadf['Open'].iloc[nrows]
    High=datadf['High'].iloc[nrows]
    Low=datadf['Low'].iloc[nrows]
    if  Open > Close :
        candle=-1
    if  Open < Close :
        candle=1
    if  Open == Close :
        candle=0
####### Check MaruBozo
    
    if candle > 0:
        if ((High-Close)/(Close-Open)) < .05 and ((Open-Low)/(Close-Open))< .05:
            pattern=pattern + 'Bull Marubozo|' +'SL:' + str(Low)
    if candle < 0:
        if ((High-Open)/(Open-Close)) < .05 and ((Close-Low)/(Open-Close)) < .05:
            pattern=pattern + 'Bear Marubozo|' +'SL:' + str(High)
        
####### Check Paper Umbrella
    if candle > 0:
        if ((Open-Low)/(Close-Open)) > 2 and (High-Close)/(Close-Open) < .05:
            umbrellaFound=True
    if candle < 0:
        if ((Close-Low)/(Open-Close)) > 2 and (High-Open)/(Open-Close) < .05:
            umbrellaFound=True
    if umbrellaFound:
        if stockdf['trend'].iloc[i] == 1:
            pattern=pattern + 'Hanging Man ' +'SL:' + str(High)
        else:
            pattern=pattern + 'Hammer ' + 'SL:' + str(Low)
        
    stockdf['singleCandlePattern'].iloc[i]=pattern
    return(stockdf)
                
        
        

In [6]:
def rsiindicator(stockdf,datadf,symbol,term):
    for i in range(0,stockdf.shape[0],1):
        if stockdf['Issuer Name'].iloc[i]==symbol:
            break;
    nrows=datadf.shape[0]-1
    datadf['Diff']=datadf['Close'].diff()
    datadf['AVGU']=0
    datadf['AVGD']=0
    datadf.loc[datadf['Diff'] > 0,'AVGU' ] = datadf['Diff']
    datadf.loc[datadf['Diff'] < 0,'AVGD' ] = -datadf['Diff']
    AVGUT=datadf.iloc[nrows-1,18]/term + (term - 1)*datadf.iloc[(nrows-(term+1)):(nrows-2),18].mean()/term    
    AVGDT=datadf.iloc[nrows-1,19]/term + (term - 1)*datadf.iloc[(nrows-(term+1)):(nrows-2),19].mean()/term
#    print(datadf.columns)
    RS=AVGUT/AVGDT
    RSI=100-(100/(1+RS))
    stockdf.iloc[i,5]=RSI
    return(stockdf)

In [7]:
def getAllDaysOfWeek(year,day):  # Get days of year 
    d = datetime(2020,1,1,0,0,0)                    # January 1st
    d += timedelta(days = day - d.weekday())  # First Sunday
    while d.year == year and d < datetime.now():
        yield d
        d += timedelta(days = 7)


In [8]:
#dataCollector()

In [9]:
#markInactiveStocks()

In [10]:
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]

equityListdf=pd.DataFrame(equityList.find({'State':"A"},{'Issuer Name':1,'_id':0}))
equityListdf['trend']=0
equityListdf['daysSinceCutOver']=0
equityListdf['trendnature']=''
equityListdf['singleCandlePattern']=''
equityListdf['RSI']=0
equityListdf['Close']=0
equityListdf['sharesbought']=0
equityListdf['moneyinvested']=0    
equityListdf['profit']=0
equityListdf['Capital']=0
equityListdf['trades']=0
equityListdf['cutoverprice']=0                
cnt=0  
i=0
capital_needed=0
equityListdf['sharesbought']=0
#print(equityListdf.columns)
for dt in stockWarehouse.find({'Symbol':'SBIN','Date':{'$gte':datetime(2019,8,30)},'Close':{'$lte':1500},'Close':{'$gte':100}},{'_id':0,'Date':1},no_cursor_timeout=True).sort('Date'):
    cnt=cnt+1
#   print(symbol,cnt)
    profit=0  
    trades=0
    sell=0
    buy=0
    for symbol in equityListdf['Issuer Name']:
        for dt1 in stockWarehouse.find({'Symbol':'SBIN','Date':{'$lte':dt['Date']}},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
            refdate=dt1 # Set Reference Data as last date when SBI Stock Data was update
        for maxdate in stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
            if maxdate == refdate :  # Check if database has latest sttock info by comparing aginst SBI
                datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}}).sort('Date',-1).limit(300))# Get top 300 row of stock in iteration
                datadftmp=datadf.iloc[-1].to_numpy()
                #print(datadftmp)
                datadf.set_index('Date',inplace=True)
                datadf.sort_index(inplace=True)
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Close']=datadf.iloc[-1,1]
                tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()
                if tempnp[0,6] < 1500 and tempnp[0,6] > 100:  # Column 6 is close price 
                    equityListdf=markTrend_Crossover(equityListdf,datadf,symbol)
                    equityListdf=rsiindicator(equityListdf,datadf,symbol,15)
                    tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()
                    datadf['avgvolume30']=datadf['Volume'].rolling(300).mean()
                    datadf['avgvolume10']=datadf['Volume'].rolling(10).mean()
                    if capital_needed < 40000 and tempnp[0,5] < 40 and tempnp[0,2] < 10   and tempnp[0,1] == 1 and tempnp[0,3] == 'Expanding' and tempnp[0,7] == 0 and datadf['avgvolume30'].iloc[-1]<datadf['avgvolume10'].iloc[-1]  :
                        #print("buy",maxdate['Date'].date(),end='|')
                        equityListdf.loc[equityListdf['Issuer Name']==symbol,'sharesbought']=tempnp[0,7]+1
                        equityListdf.loc[equityListdf['Issuer Name']==symbol,'moneyinvested']=tempnp[0,8] + tempnp[0,6]
                        capital_needed=capital_needed+tempnp[0,6]
                        buy=buy+1
                    else :
                        if tempnp[0,7] > 0:
                            avgprice=tempnp[0,8]/tempnp[0,7]
                            #print('avgprice:',avgprice)
                            if (tempnp[0,5] > 70 or tempnp[0,1] == -1 or tempnp[0,3] == 'Contracting' or ((avgprice-tempnp[0,6])/avgprice) > .5 or ((tempnp[0,6]-avgprice)/avgprice) > .1):
                                loss=((tempnp[0,6] * tempnp[0,7]) - tempnp[0,8])
                                if loss < -(avgprice*.1) :
                                    loss=-(avgprice*.1)
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'profit']=tempnp[0,9] + loss
                                capital_needed=capital_needed-(tempnp[0,6] * tempnp[0,7])
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'sharesbought']=0
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'trades']=tempnp[0,11]+1
                                equityListdf.loc[equityListdf['Issuer Name']==symbol,'moneyinvested']=0
                                sell=sell+1
    
    print('Date',dt,'Profit:',equityListdf['profit'].sum(),'Capital:',capital_needed,'sellsigs:',sell,'buysigs',buy)

Date {'Date': datetime.datetime(2019, 8, 30, 0, 0)} Profit: 0 Capital: 0 sellsigs: 0 buysigs 0
Date {'Date': datetime.datetime(2019, 9, 3, 0, 0)} Profit: 0 Capital: 0 sellsigs: 0 buysigs 0
Date {'Date': datetime.datetime(2019, 9, 4, 0, 0)} Profit: 0 Capital: 0 sellsigs: 0 buysigs 0
Date {'Date': datetime.datetime(2019, 9, 5, 0, 0)} Profit: 0 Capital: 175.7 sellsigs: 0 buysigs 1
Date {'Date': datetime.datetime(2019, 9, 6, 0, 0)} Profit: 0 Capital: 175.7 sellsigs: 0 buysigs 0
Date {'Date': datetime.datetime(2019, 9, 9, 0, 0)} Profit: 0 Capital: 175.7 sellsigs: 0 buysigs 0
Date {'Date': datetime.datetime(2019, 9, 11, 0, 0)} Profit: 0 Capital: 175.7 sellsigs: 0 buysigs 0
Date {'Date': datetime.datetime(2019, 9, 12, 0, 0)} Profit: 0 Capital: 331.7 sellsigs: 0 buysigs 1
Date {'Date': datetime.datetime(2019, 9, 13, 0, 0)} Profit: 0 Capital: 524.95 sellsigs: 0 buysigs 1
Date {'Date': datetime.datetime(2019, 9, 16, 0, 0)} Profit: 0 Capital: 524.95 sellsigs: 0 buysigs 0
Date {'Date': datetime.da

Date {'Date': datetime.datetime(2019, 12, 13, 0, 0)} Profit: 88.14499999999968 Capital: 730.9000000000007 sellsigs: 0 buysigs 1
Date {'Date': datetime.datetime(2019, 12, 16, 0, 0)} Profit: 88.14499999999968 Capital: 1938.6500000000005 sellsigs: 0 buysigs 3
Date {'Date': datetime.datetime(2019, 12, 17, 0, 0)} Profit: 108.9449999999997 Capital: 2283.8500000000004 sellsigs: 1 buysigs 1
Date {'Date': datetime.datetime(2019, 12, 18, 0, 0)} Profit: 108.9449999999997 Capital: 2283.8500000000004 sellsigs: 0 buysigs 0
Date {'Date': datetime.datetime(2019, 12, 19, 0, 0)} Profit: 109.34499999999967 Capital: 3392.3999999999996 sellsigs: 2 buysigs 6
Date {'Date': datetime.datetime(2019, 12, 20, 0, 0)} Profit: 109.34499999999967 Capital: 3796.9499999999994 sellsigs: 0 buysigs 2
Date {'Date': datetime.datetime(2019, 12, 23, 0, 0)} Profit: 135.79499999999962 Capital: 3441.8999999999996 sellsigs: 2 buysigs 1
Date {'Date': datetime.datetime(2019, 12, 24, 0, 0)} Profit: 135.79499999999962 Capital: 4994.7

CursorNotFound: cursor id 6681791854831824609 not found

In [ ]:
#print(equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI'][1])
#print(equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI'][0])
#print(equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI'][0])
equityListdf.to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\result.csv")

In [26]:
print(equityListdf.columns)
print(tempnp)

Index(['Issuer Name', 'trend', 'daysSinceCutOver', 'trendnature',
       'singleCandlePattern', 'RSI', 'Close', 'sharesbought', 'moneyinvested',
       'profit', 'Capital', 'trades', 'cutoverprice'],
      dtype='object')
[['KSL' 1 48 'Contracting' '' 36.152055857253686 215.3 0 0 0 0 0 0]]


In [26]:
print(datadf.columns)
print(datadf.iloc[-1,15])

Index(['%Deliverble', 'Close', 'Deliverable Volume', 'High', 'Last', 'Low',
       'Open', 'Prev Close', 'Series', 'Symbol', 'Trades', 'Turnover', 'VWAP',
       'Volume', '_id'],
      dtype='object')


IndexError: single positional indexer is out-of-bounds

In [27]:
#print(datadf['Volume'].rolling(30).mean().iloc[-1])
#print(datadf['Volume'].iloc[-1])
#print(datadf['Volume'].tail(20))
#print(datadf.iloc[-1])
#print(datadf.iloc[-1].to_numpy())
#print(datadf.iloc[-1].to_numpy()[13])
#print(datadf.iloc[:,0])
#and tempnp[0,6] < 1000
# and datadf.iloc[-1].to_numpy()[15] < datadf.iloc[-1].to_numpy()[16]
#and (tempnp[0,6]-tempnp[0,12])/tempnp[0,6] > .05
print(fromDate)

NameError: name 'fromDate' is not defined